In [5]:
import requests
from bs4 import BeautifulSoup
import csv

In [6]:
def scrape():
    url = "http://quotes.toscrape.com/"
    page = "/page/1"
    q_data = []
    next_page = page
    visited = set()

    while next_page:
        if next_page in visited:
            print(f"Detected loop at {next_page} exiting ")
            break
        visited.add(next_page)

        urli = url + next_page
        print(f"Scrapping {url}")

        try:
            response = requests.get(urli,timeout = 5)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Request Falied {e}")
            break
        soup =  BeautifulSoup(response.text,'html.parser')
        quotes = soup.find_all("div", class_="quote")

        if not quotes:
            print(f"No quotes found {url} exiting ")
            break
        for quote in quotes:
            try:
                text=quote.find("span",class_="text").get_text(strip=True)
                author=quote.find("small",class_="author").get_text(strip=True)
                tags=[tag.get_text(strip=True) for tag in quote.find_all("a",class_="tag")]
                q_data.append({
                    "Quote":text,
                    "Author":author,
                    "Tags":",".join(tags)
                })
            except AttributeError as e:
                print(f"Error parsing quote block {e}")
                continue 
            urli=url
            next_btn= soup.find("li",class_="next")
            next_page = next_btn.find("a")["href"] if next_btn else None
    return q_data

In [7]:
def gen_csv(data, filename="quotes.csv"):
    if not data:
        print("No data to save.")
        return

    try:
        with open(filename, "w", newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=["Quote", "Author", "Tags"])
            writer.writeheader()
            writer.writerows(data)
        print(f"Saved {len(data)} quotes to {filename}")
    except Exception as e:
        print(f"Failed to save file: {e}")

In [8]:
if __name__ == "__main__":

    scraped = scrape()
    gen_csv(scraped)

Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Scrapping http://quotes.toscrape.com/
Saved 100 quotes to quotes.csv
